<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/colorGrading/tests/LPTN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Color grading via LPTN

[https://github.com/csjliang/LPTN](https://github.com/csjliang/LPTN)


In [ ]:
# we'll link the dataset from next-cloud
import os
!apt install -y davfs2  > /dev/null

nextcloud = "/content/nextcloud"
Nextcloud_URL = "https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/" #@param {type:"string"}

os.makedirs(nextcloud, exist_ok=True)


if os.path.isfile("/etc/fstab"):
  os.remove("/etc/fstab")
with open("/etc/fstab" , "a") as f:
  f.write(Nextcloud_URL + " " + nextcloud + " davfs user,rw,auto 0 0")
!mount {nextcloud}

# open existing text file
txt = "/etc/fstab"
viewTxt = open(txt, "r")
txtContent = viewTxt.read()
print("txtContent: {}".format(txtContent))

## 2. create I/O folders

In [2]:
import shutil
from google.colab import files

def create_io(database, topic, library):
  # clean and rebuild the image folders
  input_folder = database+'/'+topic
  # '/content/nextcloud/colorGrading'
  # if os.path.exists(input_folder):
  #   shutil.rmtree(input_folder)
  # os.makedirs(input_folder)

  output_folder = database+'/'+topic+'/'+library# '/content/nextcloud/colorGrading/output'
  if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
  os.makedirs(output_folder)
  return input_folder, output_folder

input_folder, output_folder = create_io('/content/nextcloud','colorGrading','LPTN')

In [ ]:
os.listdir(input_folder)

In [4]:
print(output_folder)

/content/nextcloud/colorGrading/LPTN


In [5]:
%ls

nextcloud/  sample_data/


## 3. Clone the Git repository

In [ ]:
# clone the repository
%cd /content
if not os.path.exists('LPTN'):
  !git clone https://github.com/csjliang/LPTN
%cd LPTN/

## 4. install dependencies

In [ ]:
!pip install -r requirement.txt

## 5. Pretrained model download

In [ ]:
# dowload the pre-trained model for colorgrading
%cd /content/LPTN/experiments/

# check first if exist, otherwise create path
pretrained_pth = 'pretrained_models/net_g_FiveK_numhigh3.pth'

if not os.path.exists(pretrained_pth):
  !gdown --id 11yuFgHqZe9e4OheJ9YHp9lS-4kQC1Fml \
          -O pretrained_models/net_g_FiveK_numhigh3.pth

## 6. install pyYAML and update the YAML file with correct path for pretrained_models

In [ ]:
!pip install pyYAML

import yaml

# dowload the pre-trained model for colorgrading
%cd /content/LPTN/options/test/LPTN/
 
def read_yaml():
    """ A function to read YAML file"""
    with open('test_FiveK.yml') as f:
        config = yaml.safe_load(f)
 
    return config

read_yaml()

## 7. dataset downloaden

In [ ]:
#PYTHONPATH="./:${PYTHONPATH}" python scripts/data_preparation/download_datasets.py
/content/LPTN/scripts/data_preparation/download_datasets.py
#PYTHONPATH="./:${PYTHONPATH}" python scripts/data_preparation/create_lmdb.py
/content/LPTN/scripts/data_preparation/create_lmdb.py

## 8. Evaluation

In [ ]:
# PYTHONPATH="./:${PYTHONPATH}" CUDA_VISIBLE_DEVICES=0 python codes/test.py -opt options/test/LPTN/test_FiveK.yml
! /content/LPTN/codes/test.py -opt options/test/LPTN/test_FiveK.yml

# PYTHONPATH="./:${PYTHONPATH}" CUDA_VISIBLE_DEVICES=0 python codes/test_speed.py -opt options/test/LPTN/test_speed_FiveK.yml
! /content/LPTN/codes/test_speed.py -opt options/test/LPTN/test_speed_FiveK.yml